In [37]:
import sys
sys.path.append('../src')
from models import *
from strategies import *
from custom_datasets import *
import numpy as np
np.random.seed(0)
import tqdm

import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
import torch
torch.cuda.empty_cache()
import torch.nn as nn

from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
import pandas as pd

import time
import json

import threading



In [38]:
main_path = "/root/Master_Thesis/"
dataframes_path = main_path + "data/dataframes/"
sam_path = main_path + "sam/sam_vit_h_4b8939.pth"
expirements_path = main_path+"expirements/"
models_path = main_path + "scripts/notebooks/trained_models/"

In [39]:
df_name = "brain_df"
train_df = pd.read_csv(dataframes_path+"brain_df_train.csv")
test_df = pd.read_csv(dataframes_path+"brain_df_test.csv")

In [40]:
params = {'n_epoch': 35,
          'train_args':{'batch_size': 4, 'num_workers': 1},
          'test_args':{'batch_size': 512, 'num_workers': 1},
          'optimizer_args':{'lr': 5e-3, 'momentum': 0.9},
          'use_sam': True,
          'use_predictor': True,
          'use_generator': False,
          'init_set_size': 100,
          'query_num': 5, #int(0.1*len(test_df)),
          'rounds': 21,
          "activate_sam_at_round":1, 
          "img_size":(128, 128)}
          
params['test_set_size'] = len(test_df)
params['df'] = df_name
params["strategy"] = "MarginSampling"
params["voters"] = f'trained_models/voters/voters_{params["img_size"][0]}_'
init_main_state_path = f'trained_models/voters/voters_{params["img_size"][0]}_0'

In [41]:
def get_data(handler, train_df, test_df):
    return Data(train_df["images"].to_list(), train_df["masks"].to_list(), test_df["images"].to_list(), test_df["masks"].to_list(), handler, img_size=params["img_size"], df=train_df, path= main_path+"/data/processed/", use_sam=params['use_sam'])

In [42]:
data = get_data(Handler, train_df, test_df)
data.initialize_labels(params["init_set_size"])
results=[]

In [43]:
# for i in [1,2,3,4,5,6,7,8,9,10]:
#     model = smp.create_model(
#             'Unet', encoder_name='resnet34', in_channels=3, classes = 1
#         )
#     torch.save(model.state_dict(), f"trained_models/voters/voters_128_0/model_{i}.pt")
#     print(f"Model_{i}'s training saved!")

In [44]:

def ensemble(models_num, starting_index, params, data, cuurent_round=1, query_idxs=None):
    for i in range(starting_index, models_num+starting_index):
        print(f"Model_{i}'s training started!", flush=True)
        model = smp.create_model(
                'Unet', encoder_name='resnet34', in_channels=3, classes = 1
            )
        init_state_Unet = torch.load(f"trained_models/voters/voters_128_0/model_{i}.pt")
        net = Net(model, params, device = torch.device("cuda"))
        net.net.load_state_dict(init_state_Unet)
        strategy = MarginSampling(dataset=data, net=net, sam=None, params=params)
        if not query_idxs is None:
            strategy.update(query_idxs)
        
        strategy.train()
        torch.save( strategy.net.net.state_dict(), f'{params["voters"]}{cuurent_round}/model_{i}.pt')
        logits, mask_gt = strategy.predict(data.get_test_data())
        iou_score, accuracy, precision, recall, f1_score = data.cal_test_metrics(logits, mask_gt )
        print(f"Testing metrics for model_{i}: iou_score = {iou_score:.2f}, accuracy = {accuracy:.2f}, precision = {precision:.2f}, recall = {recall:.2f}, f1_score = {f1_score:.2f}", flush=True)
        print(f"Model_{i}'s saved!", flush=True)
    # print("Done!")
    

In [45]:
query_idxs = [1656,  121,  253,  968, 2095]

In [46]:
# ensemble(models_num=1, starting_index=3, params=params, data=data, cuurent_round=1, query_idxs=query_idxs)

In [47]:
# # for i in range(1, 11):
# for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
#     t = threading.Thread(target=ensemble, daemon=True, args=[1, i, params, data, 1, query_idxs])
#     t.start()

In [48]:
# t1 = threading.Thread(target=ensemble, daemon=True, args=[5, 1, params, data, query_idxs])
# t1.start()

In [49]:
# t2 = threading.Thread(target=ensemble, daemon=True, args=[5, 6, params, data, query_idxs])
# t2.start()

In [50]:
model = smp.create_model(
        'Unet', encoder_name='resnet34', in_channels=3, classes = 1
    )
net = Net(model, params, device = torch.device("cuda"))

In [51]:
No_Active_128_Unet = main_path + "scripts/notebooks/trained_models/No_Active_128_Unet.pt"
not_voters_Unet_128_done = main_path + "scripts/notebooks/trained_models/not_pre_trained_Unet_128_done.pt"

In [52]:
def test(net, idx, params, data, current_round=1):
    for i in idx:
        net.net.load_state_dict(torch.load(f'{params["voters"]}{current_round}/model_{i}.pt'))
        logits, mask_gt = net.predict(data.get_test_data())
        iou_score, accuracy, precision, recall, f1_score = data.cal_test_metrics(logits, mask_gt )
        print((f"Testing metrics for model_{i}: iou_score = {iou_score:.2f}, accuracy = {accuracy:.2f}, precision = {precision:.2f}, recall = {recall:.2f}, f1_score = {f1_score:.2f}"))
    

In [53]:
idx = [i for i in range(1,11)]
idx

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [54]:
test(net, idx, params, data, current_round=0)

Testing metrics for model_1: iou_score = 0.01, accuracy = 0.72, precision = 0.01, recall = 0.19, f1_score = 0.01
Testing metrics for model_2: iou_score = 0.01, accuracy = 0.08, precision = 0.01, recall = 0.96, f1_score = 0.02
Testing metrics for model_3: iou_score = 0.00, accuracy = 0.97, precision = 0.00, recall = 0.00, f1_score = 0.00
Testing metrics for model_4: iou_score = 0.01, accuracy = 0.43, precision = 0.01, recall = 0.64, f1_score = 0.02
Testing metrics for model_5: iou_score = 0.01, accuracy = 0.11, precision = 0.01, recall = 0.87, f1_score = 0.02
Testing metrics for model_6: iou_score = 0.01, accuracy = 0.01, precision = 0.01, recall = 1.00, f1_score = 0.02
Testing metrics for model_7: iou_score = 0.01, accuracy = 0.90, precision = 0.01, recall = 0.05, f1_score = 0.01
Testing metrics for model_8: iou_score = 0.01, accuracy = 0.85, precision = 0.01, recall = 0.09, f1_score = 0.01
Testing metrics for model_9: iou_score = 0.01, accuracy = 0.01, precision = 0.01, recall = 1.00,

In [37]:
test(net, idx, params, data, current_round=2)

Testing metrics for model_1: iou_score = 0.47, accuracy = 0.99, precision = 0.64, recall = 0.65, f1_score = 0.64
Testing metrics for model_2: iou_score = 0.50, accuracy = 0.99, precision = 0.61, recall = 0.73, f1_score = 0.67
Testing metrics for model_3: iou_score = 0.53, accuracy = 0.99, precision = 0.69, recall = 0.70, f1_score = 0.70
Testing metrics for model_4: iou_score = 0.52, accuracy = 0.99, precision = 0.69, recall = 0.67, f1_score = 0.68
Testing metrics for model_5: iou_score = 0.44, accuracy = 0.99, precision = 0.72, recall = 0.52, f1_score = 0.61
Testing metrics for model_6: iou_score = 0.47, accuracy = 0.99, precision = 0.64, recall = 0.64, f1_score = 0.64
Testing metrics for model_7: iou_score = 0.55, accuracy = 0.99, precision = 0.68, recall = 0.73, f1_score = 0.71
Testing metrics for model_8: iou_score = 0.45, accuracy = 0.99, precision = 0.75, recall = 0.54, f1_score = 0.62
Testing metrics for model_9: iou_score = 0.47, accuracy = 0.99, precision = 0.53, recall = 0.81,

In [38]:
test(net, idx, params, data, current_round=3)

Testing metrics for model_1: iou_score = 0.47, accuracy = 0.99, precision = 0.77, recall = 0.55, f1_score = 0.64
Testing metrics for model_2: iou_score = 0.45, accuracy = 0.99, precision = 0.81, recall = 0.51, f1_score = 0.63
Testing metrics for model_3: iou_score = 0.50, accuracy = 0.99, precision = 0.80, recall = 0.57, f1_score = 0.66
Testing metrics for model_4: iou_score = 0.51, accuracy = 0.99, precision = 0.72, recall = 0.63, f1_score = 0.68
Testing metrics for model_5: iou_score = 0.53, accuracy = 0.99, precision = 0.64, recall = 0.75, f1_score = 0.69
Testing metrics for model_6: iou_score = 0.54, accuracy = 0.99, precision = 0.64, recall = 0.77, f1_score = 0.70
Testing metrics for model_7: iou_score = 0.44, accuracy = 0.99, precision = 0.79, recall = 0.50, f1_score = 0.62
Testing metrics for model_8: iou_score = 0.45, accuracy = 0.99, precision = 0.71, recall = 0.56, f1_score = 0.62
Testing metrics for model_9: iou_score = 0.49, accuracy = 0.99, precision = 0.81, recall = 0.56,

In [39]:
test(net, idx, params, data, current_round=4)

Testing metrics for model_1: iou_score = 0.49, accuracy = 0.99, precision = 0.58, recall = 0.77, f1_score = 0.66
Testing metrics for model_2: iou_score = 0.55, accuracy = 0.99, precision = 0.74, recall = 0.68, f1_score = 0.71
Testing metrics for model_3: iou_score = 0.54, accuracy = 0.99, precision = 0.82, recall = 0.61, f1_score = 0.70
Testing metrics for model_4: iou_score = 0.52, accuracy = 0.99, precision = 0.71, recall = 0.66, f1_score = 0.69
Testing metrics for model_5: iou_score = 0.46, accuracy = 0.99, precision = 0.86, recall = 0.49, f1_score = 0.63
Testing metrics for model_6: iou_score = 0.50, accuracy = 0.99, precision = 0.69, recall = 0.64, f1_score = 0.66
Testing metrics for model_7: iou_score = 0.50, accuracy = 0.99, precision = 0.68, recall = 0.64, f1_score = 0.66
Testing metrics for model_8: iou_score = 0.48, accuracy = 0.99, precision = 0.75, recall = 0.57, f1_score = 0.65
Testing metrics for model_9: iou_score = 0.54, accuracy = 0.99, precision = 0.71, recall = 0.70,

In [40]:
test(net, idx, params, data, current_round=5)

Testing metrics for model_1: iou_score = 0.50, accuracy = 0.99, precision = 0.59, recall = 0.76, f1_score = 0.67
Testing metrics for model_2: iou_score = 0.44, accuracy = 0.99, precision = 0.85, recall = 0.48, f1_score = 0.61
Testing metrics for model_3: iou_score = 0.51, accuracy = 0.99, precision = 0.78, recall = 0.59, f1_score = 0.67
Testing metrics for model_4: iou_score = 0.54, accuracy = 0.99, precision = 0.80, recall = 0.63, f1_score = 0.70
Testing metrics for model_5: iou_score = 0.53, accuracy = 0.99, precision = 0.72, recall = 0.68, f1_score = 0.70
Testing metrics for model_6: iou_score = 0.56, accuracy = 0.99, precision = 0.71, recall = 0.72, f1_score = 0.72
Testing metrics for model_7: iou_score = 0.47, accuracy = 0.99, precision = 0.88, recall = 0.51, f1_score = 0.64
Testing metrics for model_8: iou_score = 0.55, accuracy = 0.99, precision = 0.69, recall = 0.73, f1_score = 0.71
Testing metrics for model_9: iou_score = 0.53, accuracy = 0.99, precision = 0.79, recall = 0.62,

In [41]:
test(net, idx, params, data, current_round=6)

Testing metrics for model_1: iou_score = 0.45, accuracy = 0.99, precision = 0.83, recall = 0.50, f1_score = 0.63
Testing metrics for model_2: iou_score = 0.54, accuracy = 0.99, precision = 0.78, recall = 0.63, f1_score = 0.70
Testing metrics for model_3: iou_score = 0.54, accuracy = 0.99, precision = 0.74, recall = 0.66, f1_score = 0.70
Testing metrics for model_4: iou_score = 0.51, accuracy = 0.99, precision = 0.63, recall = 0.73, f1_score = 0.68
Testing metrics for model_5: iou_score = 0.55, accuracy = 0.99, precision = 0.68, recall = 0.74, f1_score = 0.71
Testing metrics for model_6: iou_score = 0.54, accuracy = 0.99, precision = 0.68, recall = 0.71, f1_score = 0.70
Testing metrics for model_7: iou_score = 0.51, accuracy = 0.99, precision = 0.82, recall = 0.58, f1_score = 0.68
Testing metrics for model_8: iou_score = 0.57, accuracy = 0.99, precision = 0.71, recall = 0.74, f1_score = 0.72
Testing metrics for model_9: iou_score = 0.56, accuracy = 0.99, precision = 0.74, recall = 0.70,

In [42]:
test(net, idx, params, data, current_round=7)

Testing metrics for model_1: iou_score = 0.51, accuracy = 0.99, precision = 0.80, recall = 0.58, f1_score = 0.67
Testing metrics for model_2: iou_score = 0.54, accuracy = 0.99, precision = 0.82, recall = 0.61, f1_score = 0.70
Testing metrics for model_3: iou_score = 0.55, accuracy = 0.99, precision = 0.75, recall = 0.68, f1_score = 0.71
Testing metrics for model_4: iou_score = 0.57, accuracy = 0.99, precision = 0.76, recall = 0.69, f1_score = 0.72
Testing metrics for model_5: iou_score = 0.54, accuracy = 0.99, precision = 0.74, recall = 0.66, f1_score = 0.70
Testing metrics for model_6: iou_score = 0.52, accuracy = 0.99, precision = 0.82, recall = 0.58, f1_score = 0.68
Testing metrics for model_7: iou_score = 0.55, accuracy = 0.99, precision = 0.76, recall = 0.66, f1_score = 0.71
Testing metrics for model_8: iou_score = 0.52, accuracy = 0.99, precision = 0.84, recall = 0.57, f1_score = 0.68
Testing metrics for model_9: iou_score = 0.50, accuracy = 0.99, precision = 0.81, recall = 0.57,

In [43]:
test(net, idx, params, data, current_round=8)

Testing metrics for model_1: iou_score = 0.55, accuracy = 0.99, precision = 0.76, recall = 0.67, f1_score = 0.71
Testing metrics for model_2: iou_score = 0.54, accuracy = 0.99, precision = 0.83, recall = 0.61, f1_score = 0.70
Testing metrics for model_3: iou_score = 0.48, accuracy = 0.99, precision = 0.78, recall = 0.55, f1_score = 0.65
Testing metrics for model_4: iou_score = 0.50, accuracy = 0.99, precision = 0.82, recall = 0.56, f1_score = 0.67
Testing metrics for model_5: iou_score = 0.54, accuracy = 0.99, precision = 0.70, recall = 0.69, f1_score = 0.70
Testing metrics for model_6: iou_score = 0.45, accuracy = 0.99, precision = 0.54, recall = 0.72, f1_score = 0.62
Testing metrics for model_7: iou_score = 0.48, accuracy = 0.99, precision = 0.86, recall = 0.52, f1_score = 0.64
Testing metrics for model_8: iou_score = 0.51, accuracy = 0.99, precision = 0.81, recall = 0.57, f1_score = 0.67
Testing metrics for model_9: iou_score = 0.53, accuracy = 0.99, precision = 0.86, recall = 0.58,

In [44]:
test(net, idx, params, data, current_round=9)

Testing metrics for model_1: iou_score = 0.55, accuracy = 0.99, precision = 0.81, recall = 0.63, f1_score = 0.71
Testing metrics for model_2: iou_score = 0.51, accuracy = 0.99, precision = 0.86, recall = 0.56, f1_score = 0.68
Testing metrics for model_3: iou_score = 0.51, accuracy = 0.99, precision = 0.78, recall = 0.59, f1_score = 0.68
Testing metrics for model_4: iou_score = 0.50, accuracy = 0.99, precision = 0.71, recall = 0.63, f1_score = 0.67
Testing metrics for model_5: iou_score = 0.52, accuracy = 0.99, precision = 0.82, recall = 0.59, f1_score = 0.69
Testing metrics for model_6: iou_score = 0.49, accuracy = 0.99, precision = 0.73, recall = 0.59, f1_score = 0.66
Testing metrics for model_7: iou_score = 0.53, accuracy = 0.99, precision = 0.81, recall = 0.61, f1_score = 0.70
Testing metrics for model_8: iou_score = 0.56, accuracy = 0.99, precision = 0.83, recall = 0.63, f1_score = 0.72
Testing metrics for model_9: iou_score = 0.50, accuracy = 0.99, precision = 0.84, recall = 0.56,

In [45]:
test(net, idx, params, data, current_round=10)

Testing metrics for model_1: iou_score = 0.58, accuracy = 0.99, precision = 0.76, recall = 0.71, f1_score = 0.73
Testing metrics for model_2: iou_score = 0.56, accuracy = 0.99, precision = 0.80, recall = 0.65, f1_score = 0.72
Testing metrics for model_3: iou_score = 0.56, accuracy = 0.99, precision = 0.77, recall = 0.67, f1_score = 0.72
Testing metrics for model_4: iou_score = 0.57, accuracy = 0.99, precision = 0.74, recall = 0.70, f1_score = 0.72
Testing metrics for model_5: iou_score = 0.50, accuracy = 0.99, precision = 0.87, recall = 0.54, f1_score = 0.67
Testing metrics for model_6: iou_score = 0.50, accuracy = 0.99, precision = 0.90, recall = 0.53, f1_score = 0.67
Testing metrics for model_7: iou_score = 0.55, accuracy = 0.99, precision = 0.84, recall = 0.61, f1_score = 0.71
Testing metrics for model_8: iou_score = 0.53, accuracy = 0.99, precision = 0.79, recall = 0.62, f1_score = 0.69
Testing metrics for model_9: iou_score = 0.52, accuracy = 0.99, precision = 0.85, recall = 0.58,

In [46]:
test(net, idx, params, data, current_round=11)

Testing metrics for model_1: iou_score = 0.51, accuracy = 0.99, precision = 0.83, recall = 0.57, f1_score = 0.67
Testing metrics for model_2: iou_score = 0.55, accuracy = 0.99, precision = 0.83, recall = 0.62, f1_score = 0.71
Testing metrics for model_3: iou_score = 0.49, accuracy = 0.99, precision = 0.86, recall = 0.54, f1_score = 0.66
Testing metrics for model_4: iou_score = 0.58, accuracy = 0.99, precision = 0.83, recall = 0.66, f1_score = 0.74
Testing metrics for model_5: iou_score = 0.52, accuracy = 0.99, precision = 0.88, recall = 0.56, f1_score = 0.69
Testing metrics for model_6: iou_score = 0.57, accuracy = 0.99, precision = 0.76, recall = 0.69, f1_score = 0.72
Testing metrics for model_7: iou_score = 0.53, accuracy = 0.99, precision = 0.87, recall = 0.57, f1_score = 0.69
Testing metrics for model_8: iou_score = 0.53, accuracy = 0.99, precision = 0.82, recall = 0.60, f1_score = 0.70
Testing metrics for model_9: iou_score = 0.54, accuracy = 0.99, precision = 0.76, recall = 0.65,

In [47]:
test(net, idx, params, data, current_round=12)

Testing metrics for model_1: iou_score = 0.50, accuracy = 0.99, precision = 0.87, recall = 0.55, f1_score = 0.67
Testing metrics for model_2: iou_score = 0.51, accuracy = 0.99, precision = 0.85, recall = 0.56, f1_score = 0.67
Testing metrics for model_3: iou_score = 0.57, accuracy = 0.99, precision = 0.78, recall = 0.68, f1_score = 0.72
Testing metrics for model_4: iou_score = 0.58, accuracy = 0.99, precision = 0.80, recall = 0.68, f1_score = 0.73
Testing metrics for model_5: iou_score = 0.56, accuracy = 0.99, precision = 0.68, recall = 0.76, f1_score = 0.72
Testing metrics for model_6: iou_score = 0.49, accuracy = 0.99, precision = 0.70, recall = 0.62, f1_score = 0.66
Testing metrics for model_7: iou_score = 0.58, accuracy = 0.99, precision = 0.86, recall = 0.64, f1_score = 0.73
Testing metrics for model_8: iou_score = 0.57, accuracy = 0.99, precision = 0.82, recall = 0.65, f1_score = 0.72
Testing metrics for model_9: iou_score = 0.55, accuracy = 0.99, precision = 0.82, recall = 0.62,

In [48]:
test(net, idx, params, data, current_round=13)

Testing metrics for model_1: iou_score = 0.56, accuracy = 0.99, precision = 0.82, recall = 0.65, f1_score = 0.72
Testing metrics for model_2: iou_score = 0.56, accuracy = 0.99, precision = 0.71, recall = 0.71, f1_score = 0.71
Testing metrics for model_3: iou_score = 0.54, accuracy = 0.99, precision = 0.78, recall = 0.63, f1_score = 0.70
Testing metrics for model_4: iou_score = 0.54, accuracy = 0.99, precision = 0.63, recall = 0.80, f1_score = 0.70
Testing metrics for model_5: iou_score = 0.51, accuracy = 0.99, precision = 0.85, recall = 0.56, f1_score = 0.68
Testing metrics for model_6: iou_score = 0.55, accuracy = 0.99, precision = 0.75, recall = 0.68, f1_score = 0.71
Testing metrics for model_7: iou_score = 0.54, accuracy = 0.99, precision = 0.65, recall = 0.77, f1_score = 0.70
Testing metrics for model_8: iou_score = 0.55, accuracy = 0.99, precision = 0.74, recall = 0.68, f1_score = 0.71
Testing metrics for model_9: iou_score = 0.53, accuracy = 0.99, precision = 0.82, recall = 0.60,

In [49]:
test(net, idx, params, data, current_round=14)

Testing metrics for model_1: iou_score = 0.58, accuracy = 0.99, precision = 0.80, recall = 0.68, f1_score = 0.73
Testing metrics for model_2: iou_score = 0.51, accuracy = 0.99, precision = 0.85, recall = 0.56, f1_score = 0.67
Testing metrics for model_3: iou_score = 0.49, accuracy = 0.99, precision = 0.86, recall = 0.53, f1_score = 0.66
Testing metrics for model_4: iou_score = 0.55, accuracy = 0.99, precision = 0.81, recall = 0.64, f1_score = 0.71
Testing metrics for model_5: iou_score = 0.56, accuracy = 0.99, precision = 0.82, recall = 0.64, f1_score = 0.72
Testing metrics for model_6: iou_score = 0.56, accuracy = 0.99, precision = 0.77, recall = 0.67, f1_score = 0.72
Testing metrics for model_7: iou_score = 0.50, accuracy = 0.99, precision = 0.82, recall = 0.56, f1_score = 0.67
Testing metrics for model_8: iou_score = 0.49, accuracy = 0.99, precision = 0.88, recall = 0.53, f1_score = 0.66
Testing metrics for model_9: iou_score = 0.56, accuracy = 0.99, precision = 0.85, recall = 0.63,

In [50]:
test(net, idx, params, data, current_round=15)

Testing metrics for model_1: iou_score = 0.58, accuracy = 0.99, precision = 0.82, recall = 0.67, f1_score = 0.74
Testing metrics for model_2: iou_score = 0.60, accuracy = 1.00, precision = 0.84, recall = 0.68, f1_score = 0.75
Testing metrics for model_3: iou_score = 0.53, accuracy = 0.99, precision = 0.85, recall = 0.58, f1_score = 0.69
Testing metrics for model_4: iou_score = 0.49, accuracy = 0.99, precision = 0.85, recall = 0.54, f1_score = 0.66
Testing metrics for model_5: iou_score = 0.59, accuracy = 0.99, precision = 0.81, recall = 0.69, f1_score = 0.74
Testing metrics for model_6: iou_score = 0.56, accuracy = 0.99, precision = 0.68, recall = 0.75, f1_score = 0.71
Testing metrics for model_7: iou_score = 0.49, accuracy = 0.99, precision = 0.82, recall = 0.54, f1_score = 0.65
Testing metrics for model_8: iou_score = 0.56, accuracy = 0.99, precision = 0.72, recall = 0.72, f1_score = 0.72
Testing metrics for model_9: iou_score = 0.50, accuracy = 0.99, precision = 0.82, recall = 0.56,

In [51]:
test(net, idx, params, data, current_round=16)

Testing metrics for model_1: iou_score = 0.58, accuracy = 0.99, precision = 0.76, recall = 0.71, f1_score = 0.73
Testing metrics for model_2: iou_score = 0.58, accuracy = 0.99, precision = 0.83, recall = 0.66, f1_score = 0.73
Testing metrics for model_3: iou_score = 0.58, accuracy = 0.99, precision = 0.80, recall = 0.68, f1_score = 0.74
Testing metrics for model_4: iou_score = 0.53, accuracy = 0.99, precision = 0.84, recall = 0.60, f1_score = 0.70
Testing metrics for model_5: iou_score = 0.60, accuracy = 1.00, precision = 0.82, recall = 0.69, f1_score = 0.75
Testing metrics for model_6: iou_score = 0.56, accuracy = 0.99, precision = 0.74, recall = 0.70, f1_score = 0.72
Testing metrics for model_7: iou_score = 0.59, accuracy = 1.00, precision = 0.86, recall = 0.65, f1_score = 0.74
Testing metrics for model_8: iou_score = 0.57, accuracy = 0.99, precision = 0.76, recall = 0.70, f1_score = 0.73
Testing metrics for model_9: iou_score = 0.60, accuracy = 0.99, precision = 0.74, recall = 0.76,

In [52]:
test(net, idx, params, data, current_round=17)

Testing metrics for model_1: iou_score = 0.59, accuracy = 0.99, precision = 0.79, recall = 0.70, f1_score = 0.74
Testing metrics for model_2: iou_score = 0.58, accuracy = 0.99, precision = 0.83, recall = 0.65, f1_score = 0.73
Testing metrics for model_3: iou_score = 0.54, accuracy = 0.99, precision = 0.81, recall = 0.61, f1_score = 0.70
Testing metrics for model_4: iou_score = 0.58, accuracy = 0.99, precision = 0.86, recall = 0.64, f1_score = 0.73
Testing metrics for model_5: iou_score = 0.57, accuracy = 0.99, precision = 0.86, recall = 0.63, f1_score = 0.73
Testing metrics for model_6: iou_score = 0.54, accuracy = 0.99, precision = 0.85, recall = 0.60, f1_score = 0.70
Testing metrics for model_7: iou_score = 0.58, accuracy = 0.99, precision = 0.82, recall = 0.66, f1_score = 0.73
Testing metrics for model_8: iou_score = 0.56, accuracy = 0.99, precision = 0.86, recall = 0.62, f1_score = 0.72
Testing metrics for model_9: iou_score = 0.54, accuracy = 0.99, precision = 0.85, recall = 0.60,

In [53]:
test(net, idx, params, data, current_round=18)

Testing metrics for model_1: iou_score = 0.56, accuracy = 0.99, precision = 0.79, recall = 0.66, f1_score = 0.72
Testing metrics for model_2: iou_score = 0.56, accuracy = 0.99, precision = 0.72, recall = 0.72, f1_score = 0.72
Testing metrics for model_3: iou_score = 0.61, accuracy = 1.00, precision = 0.80, recall = 0.72, f1_score = 0.76
Testing metrics for model_4: iou_score = 0.61, accuracy = 1.00, precision = 0.83, recall = 0.70, f1_score = 0.76
Testing metrics for model_5: iou_score = 0.59, accuracy = 0.99, precision = 0.78, recall = 0.71, f1_score = 0.74
Testing metrics for model_6: iou_score = 0.58, accuracy = 0.99, precision = 0.76, recall = 0.71, f1_score = 0.74
Testing metrics for model_7: iou_score = 0.56, accuracy = 0.99, precision = 0.82, recall = 0.64, f1_score = 0.72
Testing metrics for model_8: iou_score = 0.56, accuracy = 0.99, precision = 0.81, recall = 0.65, f1_score = 0.72
Testing metrics for model_9: iou_score = 0.59, accuracy = 0.99, precision = 0.80, recall = 0.69,

In [54]:
test(net, idx, params, data, current_round=19)

Testing metrics for model_1: iou_score = 0.58, accuracy = 0.99, precision = 0.76, recall = 0.71, f1_score = 0.73
Testing metrics for model_2: iou_score = 0.59, accuracy = 0.99, precision = 0.82, recall = 0.67, f1_score = 0.74
Testing metrics for model_3: iou_score = 0.62, accuracy = 1.00, precision = 0.82, recall = 0.72, f1_score = 0.76
Testing metrics for model_4: iou_score = 0.58, accuracy = 0.99, precision = 0.78, recall = 0.69, f1_score = 0.73
Testing metrics for model_5: iou_score = 0.59, accuracy = 0.99, precision = 0.79, recall = 0.69, f1_score = 0.74
Testing metrics for model_6: iou_score = 0.57, accuracy = 0.99, precision = 0.78, recall = 0.68, f1_score = 0.73
Testing metrics for model_7: iou_score = 0.58, accuracy = 0.99, precision = 0.81, recall = 0.68, f1_score = 0.74
Testing metrics for model_8: iou_score = 0.60, accuracy = 0.99, precision = 0.80, recall = 0.70, f1_score = 0.75
Testing metrics for model_9: iou_score = 0.56, accuracy = 0.99, precision = 0.79, recall = 0.66,

In [55]:
test(net, idx, params, data, current_round=20)

Testing metrics for model_1: iou_score = 0.60, accuracy = 0.99, precision = 0.74, recall = 0.76, f1_score = 0.75
Testing metrics for model_2: iou_score = 0.61, accuracy = 1.00, precision = 0.81, recall = 0.71, f1_score = 0.76
Testing metrics for model_3: iou_score = 0.58, accuracy = 0.99, precision = 0.84, recall = 0.65, f1_score = 0.73
Testing metrics for model_4: iou_score = 0.60, accuracy = 0.99, precision = 0.75, recall = 0.74, f1_score = 0.75
Testing metrics for model_5: iou_score = 0.55, accuracy = 0.99, precision = 0.87, recall = 0.59, f1_score = 0.71
Testing metrics for model_6: iou_score = 0.59, accuracy = 0.99, precision = 0.77, recall = 0.72, f1_score = 0.75
Testing metrics for model_7: iou_score = 0.56, accuracy = 0.99, precision = 0.84, recall = 0.63, f1_score = 0.72
Testing metrics for model_8: iou_score = 0.60, accuracy = 0.99, precision = 0.81, recall = 0.70, f1_score = 0.75
Testing metrics for model_9: iou_score = 0.59, accuracy = 0.99, precision = 0.74, recall = 0.75,